In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('../artifacts/data_ingestion/winequality-red.csv')

In [5]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [6]:
df.shape

(1599, 12)

In [7]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [8]:
df['quality'].unique()

array([5, 6, 7, 4, 8, 3])

In [12]:
columns = df.columns
columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [16]:
np.array(df['fixed acidity'])

array([7.4, 7.8, 7.8, ..., 6.3, 5.9, 6. ], shape=(1599,))

In [13]:
for col in columns:
    print(col)

fixed acidity
volatile acidity
citric acid
residual sugar
chlorides
free sulfur dioxide
total sulfur dioxide
density
pH
sulphates
alcohol
quality


In [45]:
def compute_valid_input_ranges(df):
    input_value_range = {}
    for col in df.columns:
        data = df[col].values
        Q1 = np.percentile(data, 25)
        Q3 = np.percentile(data, 75)
        IQR = Q3 - Q1
        
        lower_bound = round(float(Q1 - 1.5 * IQR), 2)
        upper_bound = round(float(Q3 + 1.5 * IQR), 2)
        
        # Fix negative lower bounds for selected columns
        if col in ['citric acid', 'free sulfur dioxide', 'total sulfur dioxide']:
            lower_bound = float(max(lower_bound, df[col].min()))
        
        input_value_range[col] = [lower_bound, upper_bound]
        
    return input_value_range

In [46]:
compute_valid_input_ranges(df)

{'fixed acidity': [3.95, 12.35],
 'volatile acidity': [0.02, 1.02],
 'citric acid': [0.0, 0.91],
 'residual sugar': [0.85, 3.65],
 'chlorides': [0.04, 0.12],
 'free sulfur dioxide': [1.0, 42.0],
 'total sulfur dioxide': [6.0, 122.0],
 'density': [0.99, 1.0],
 'pH': [2.92, 3.68],
 'sulphates': [0.28, 1.0],
 'alcohol': [7.1, 13.5],
 'quality': [3.5, 7.5]}

These IQR ranges are based only on the dataset we have —but in the real world, pH or alcohol etc. can be slightly outside this range. How do we handle that?
IQR ranges are NOT a representation of the true physical limits of wine. They only reflect your dataset’s distribution.

Some values are negative (citric acid: -0.4, free sulfur dioxide: -14.0, total sulfur dioxide: -38.0). These don’t make sense physically (negative acidity or sulfur values aren’t valid). This happens because IQR method is purely statistical.

Combine IQR-based ranges + scientific constraints. That gives us robust yet realistic validation.

In [56]:
valid_input_ranges = {
    'fixed_acidity': [3.95, 12.35],
    'volatile_acidity': [0.02, 1.02],
    'citric_acid': [0.0, 0.91],
    'residual_sugar': [0.85, 3.65],
    'chlorides': [0.04, 0.12],
    'free_sulfur_dioxide': [1.0, 42.0],
    'total_sulfur_dioxide': [6.0, 122.0],
    'density': [0.99, 1.0],
    'pH': [2.92, 3.68],
    'sulphates': [0.28, 1.0],
    'alcohol': [7.1, 13.5]
}

In [57]:
inputs = {
    'fixed_acidity': 4,
    'volatile_acidity': 0.9,
    'citric_acid': 0.4,
    'residual_sugar': 2,
    'chlorides': 0.1,
    'free_sulfur_dioxide': 41,
    'total_sulfur_dioxide': 57,
    'density': 0.993,
    'pH': 0.83,
    'sulphates': 0.9,
    'alcohol': 7 
}

In [58]:
for feature, value in inputs.items():
    min_val, max_val = valid_input_ranges[feature]
    if value < min_val:
        inputs[feature] = min_val
    elif value > max_val:
        inputs[feature] = max_val